In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [31]:
ds = pd.read_csv('run_race.csv')
ds.head()

,race_id,country,horse_type,horse_age,horse_place,horse_weight,rider_weight,average_speed,distance,going,config,surface,horse_ratings,venue,winning_horse_speed
0,1,IRE,Gelding,4,1,1027.0,126.0,17.290184,1600,Good,A,0,60-80,HV,17.495082
1,1,IRE,Mount,8,10,1156.0,141.0,15.961113,1600,Good,A,0,60-80,HV,17.495082
2,1,GB,Horse,4,2,975.0,129.0,16.801852,1600,Good,A,0,60-80,HV,17.495082
3,1,USA,Gelding,3,3,1018.0,128.0,16.759707,1600,Good,A,0,60-80,HV,17.495082
4,1,IRE,Colt,6,4,1086.0,133.0,16.593614,1600,Good,A,0,60-80,HV,17.495082


In [32]:
ds.isnull().any()

race_id                False
country                False
horse_type              True
horse_age              False
horse_place            False
horse_weight            True
rider_weight            True
average_speed          False
distance               False
going                  False
config                 False
surface                False
horse_ratings          False
venue                  False
winning_horse_speed    False
dtype: bool

In [33]:
ds.shape

(79440, 15)

In [34]:
ds = ds.dropna()
ds.shape

(79431, 15)

In [35]:
pred_speed = ds[['horse_type','config','horse_age','horse_weight','rider_weight','surface','average_speed']]
pred_speed.head()

,horse_type,config,horse_age,horse_weight,rider_weight,surface,average_speed
0,Gelding,A,4,1027.0,126.0,0,17.290184
1,Mount,A,8,1156.0,141.0,0,15.961113
2,Horse,A,4,975.0,129.0,0,16.801852
3,Gelding,A,3,1018.0,128.0,0,16.759707
4,Colt,A,6,1086.0,133.0,0,16.593614


In [32]:
#ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0,1])], remainder='passthrough')
#X = np.array(ct.fit_transform(X))

In [36]:
htype = LabelEncoder()
pred_speed['horse_type'] = htype.fit_transform(pred_speed['horse_type'])
pred_speed['horse_type'].unique()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


array([2, 5, 3, 1, 0, 6, 4])

In [37]:
config = LabelEncoder()
pred_speed['config'] = config.fit_transform(pred_speed['config'])
pred_speed['config'].unique()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


array([0, 4, 2, 3, 1])

In [38]:
pred_speed.head()

,horse_type,config,horse_age,horse_weight,rider_weight,surface,average_speed
0,2,0,4,1027.0,126.0,0,17.290184
1,5,0,8,1156.0,141.0,0,15.961113
2,3,0,4,975.0,129.0,0,16.801852
3,2,0,3,1018.0,128.0,0,16.759707
4,1,0,6,1086.0,133.0,0,16.593614


In [47]:
X = pred_speed.iloc[:,0:6].values
Y = pred_speed.iloc[:, -1].values

In [42]:
#from sklearn.model_selection import train_test_split
#X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size = 0.01, random_state=0)

In [49]:
from sklearn.linear_model import LinearRegression
linearReg = LinearRegression()
linearReg.fit(X,Y)

LinearRegression()

In [50]:
Y_pred = linearReg.predict(X)
np.set_printoptions(precision=5)
print(np.concatenate((Y_pred.reshape(len(Y_pred),1),Y.reshape(len(Y),1)),1))

[[16.92423 17.29018]
 [15.97515 15.96111]
 [16.74754 16.80185]
 ...
 [16.39458 16.40292]
 [16.35856 16.33723]
 [16.25563 16.30201]]


In [52]:
print("Accuracy: {}".format(linearReg.score(X,Y)))

Accuracy: 0.8134851741297058


In [53]:
linearReg.predict([[2,3,3,1018.0, 128.0, 0]])
      

array([16.83774])

In [55]:
X = np.array([["Gelding","A",8,1156,141,0]])
X

array([['Gelding', 'A', '8', '1156', '141', '0']], dtype='<U21')

In [56]:
X[:,0] = htype.transform(X[:,0])
X[:,1] = config.transform(X[:,1])
X = X.astype(float)
X

array([[   2.,    0.,    8., 1156.,  141.,    0.]])

In [57]:
Y_pred = linearReg.predict(X)
Y_pred

array([16.00492])

In [58]:
data = {"model" : linearReg, "horse_type" : htype, "config" : config} #Dictionary
with open('speed_predictor.pkl', 'wb') as file:
  pickle.dump(data,file)

In [60]:
with open('speed_predictor.pkl', 'rb') as file:
  pickle.load(file)

linearReg_loaded = data["model"]
htype = data["horse_type"]
config = data["config"]

In [61]:
Y_pred = linearReg_loaded.predict(X)
Y_pred

array([16.00492])